In [1]:
# Adaptive histogram equalization
import cv2
import os, ntpath
from PIL import Image, ImageOps

In [2]:
DATA_DIRECTORY = '../../data'

ORIGINAL_DATA = os.path.join(DATA_DIRECTORY, 'original', 'asl-alphabet')
FABRICATED_DATA = os.path.join(DATA_DIRECTORY, 'fabricated', 'asl_alphabet')


FABRICATED_TRAIN_DIR = os.path.join(FABRICATED_DATA, 'asl_alphabet_train')
FABRICATED_TEST_DIR = os.path.join(FABRICATED_DATA, 'asl_alphabet_test')

AHE_TRAIN_DIR = os.path.join(FABRICATED_DATA, 'ahe_asl_train')
AHE_TEST_DIR = os.path.join(FABRICATED_DATA, 'ahe_asl_test')

AJE_NEG_TRAIN_DIR = os.path.join(FABRICATED_DATA, 'ahe_neg_asl_train')
AHE_NEG_TEST_DIR = os.path.join(FABRICATED_DATA, 'ahe_neg_asl_test')

# Used to quickly create all of these directories
FABRICATED_DIRS = [
    AHE_TRAIN_DIR,
    AHE_TEST_DIR,
    AJE_NEG_TRAIN_DIR,
    AHE_NEG_TEST_DIR
]

# Used to declaritively define relatinships of where we will be saving images
DIRECTORY_MAPPINGS = {
    FABRICATED_TRAIN_DIR: { 'reg': AHE_TRAIN_DIR, 'neg': AJE_NEG_TRAIN_DIR},
    FABRICATED_TEST_DIR: { 'reg': AHE_TEST_DIR, 'neg': AHE_NEG_TEST_DIR}
}

In [3]:
# A, B, C, D, DELETE, etc
ASL_FOLDERS = os.listdir(FABRICATED_TRAIN_DIR)

In [4]:
def create_test_folders_with_letter(letter):
    for test_dir in FABRICATED_DIRS:
        test_dir_with_letter = os.path.join(test_dir, letter)
        os.makedirs(test_dir_with_letter)

In [5]:
def create_test_folders():
    # create the output folders matching the train data set folders
    successfully_created = 0
    unsuccessfully_created = 0

    for sign_name in ASL_FOLDERS:   
        try:
            create_test_folders_with_letter(sign_name)
            successfully_created = successfully_created + 1
        except:
            unsuccessfully_created = unsuccessfully_created + 1


    print('successfully created {} sets of folders'.format(successfully_created))
    print('unsuccessfully created {} sets of folders (probably already existed)'.format(unsuccessfully_created))
create_test_folders()

successfully created 29 sets of folders
unsuccessfully created 0 sets of folders (probably already existed)


In [6]:
# create enhanced images using adaptive histogram equalization (ahe)
# create negative images of the ahe images
def enhance_and_save_images():
    for from_dir in DIRECTORY_MAPPINGS:
        print('copying from {}'.format(from_dir))
        #from_dir = FABRICATED_TEST_DIR
        dir_mapping = DIRECTORY_MAPPINGS[from_dir]
        for dirs in DIRECTORY_MAPPINGS[from_dir]:
            print()
            print('copying to {}'.format(DIRECTORY_MAPPINGS[from_dir][dirs]))

        for sign_name in ASL_FOLDERS:
            in_test_dir = os.path.join(from_dir, sign_name)
            for infile in os.listdir(in_test_dir):
                out_dir = os.path.join(dir_mapping['reg'], sign_name)
                outfile = os.path.join(out_dir, infile)

                img = cv2.imread(os.path.join(in_test_dir, infile), 0)

                # create a CLAHE object (Arguments are optional).
                clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
                cl1 = clahe.apply(img)
                cv2.imwrite(outfile, cl1)

                # create an inverted (negative) image
                out_dir = os.path.join(dir_mapping['neg'], sign_name)
                outfile = os.path.join(out_dir, infile)


                #img = Image.open(os.path.join(ahe_dir, infile))
                invert_ahe = ImageOps.invert(Image.fromarray(cl1))
                invert_ahe.save(outfile)
                break
enhance_and_save_images()

copying from ../../data/fabricated/asl_alphabet/asl_alphabet_train

copying to ../../data/fabricated/asl_alphabet/ahe_asl_train

copying to ../../data/fabricated/asl_alphabet/ahe_neg_asl_train
copying from ../../data/fabricated/asl_alphabet/asl_alphabet_test

copying to ../../data/fabricated/asl_alphabet/ahe_asl_test

copying to ../../data/fabricated/asl_alphabet/ahe_neg_asl_test
